In [ ]:
streamlit
groq
pypdf
faiss-cpu
sentence-transformers
langchain
langchain-community

https://github.com/Vikneshwara-kumar/RAG_Based_Talk_to_PDF

In [ ]:
!pip install -r a.txt

شغال

In [ ]:
import os
from typing import List, Dict
from pypdf import PdfReader
from langchain_core.embeddings import Embeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

class PDFQASystem:
    def __init__(self, model_id: str):
        """Initialize the PDF QA System with necessary components."""
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")
        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2"
        )
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len,
        )
        self.vector_store = None

    def process_pdf(self, pdf_file) -> int:
        """Process a PDF file and create embeddings."""
        pdf_text = self._extract_text_from_pdf(pdf_file)
        chunks = self.text_splitter.split_text(pdf_text)
        self.vector_store = FAISS.from_texts(chunks, self.embeddings)
        return len(chunks)

    def _extract_text_from_pdf(self, pdf_file) -> str:
        """Extract text content from a PDF file."""
        reader = PdfReader(pdf_file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        return text

    def generate_response(self, user_query: str, k: int = 4) -> str:
        """Generate a response using relevant context and the LLM."""
        if not self.vector_store:
            return "Please upload a PDF document first."

        relevant_docs = self.vector_store.similarity_search(user_query, k=k)
        context = "\n".join(doc.page_content for doc in relevant_docs)

        prompt = f"""You are a helpful assistant. Using the following context, please answer the user's question as detailed as possible from the provided context, make sure to provide all the details.
        If the answer cannot be found in the context, say so.

        Context:
        {context}

        Question: {user_query}

        Please structure your response as follows:
        - **Key Points**: List each main point in bullet points.
        - **Summary**: Provide a brief paragraph summarizing the main findings.

        Answer:"""

        inputs = self.tokenizer(prompt, return_tensors="pt", max_length=2048, truncation=True).to(self.model.device)
        outputs = self.model.generate(**inputs, max_new_tokens=10)
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        return response

# Example usage in a Colab environment
def main():
    model_id = "meta-llama/Llama-3.2-3B"  # Updated model ID
    qa_system = PDFQASystem(model_id)

    pdf_path = "/content/temp.pdf"  # Replace with your PDF file path
    if not os.path.exists(pdf_path):
        print("PDF file not found!")
        return

    print("Processing PDF...")
    num_chunks = qa_system.process_pdf(pdf_path)
    print(f"PDF processed into {num_chunks} chunks.")

    while True:
        user_query = input("Enter your question (or type 'exit' to quit): ")
        if user_query.lower() == "exit":
            break

        print("Generating response...")
        response = qa_system.generate_response(user_query)
        print("\nResponse:\n")
        print(response)

if __name__ == "__main__":
    main()


In [ ]:
import os
from typing import List, Dict
from pypdf import PdfReader
from langchain_core.embeddings import Embeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

class PDFQASystem:
    def __init__(self, model_id: str):
        """Initialize the PDF QA System with necessary components."""
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")
        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2"
        )
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len,
        )
        self.vector_store = None

    def process_pdf(self, pdf_file) -> int:
        """Process a PDF file and create embeddings."""
        pdf_text = self._extract_text_from_pdf(pdf_file)
        chunks = self.text_splitter.split_text(pdf_text)
        self.vector_store = FAISS.from_texts(chunks, self.embeddings)
        return len(chunks)

    def _extract_text_from_pdf(self, pdf_file) -> str:
        """Extract text content from a PDF file."""
        reader = PdfReader(pdf_file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        return text

    def generate_response(self, user_query: str, k: int = 4) -> str:
        """Generate a response using relevant context and the LLM."""
        if not self.vector_store:
            return "Please upload a PDF document first."

        relevant_docs = self.vector_store.similarity_search(user_query, k=k)
        context = "\n".join(doc.page_content for doc in relevant_docs)

        prompt = f"""You are a helpful assistant. Using the following context, please answer the user's question as detailed as possible from the provided context, make sure to provide all the details.
        If the answer cannot be found in the context, say so.

        Context:


        Question: {user_query}

        Please structure your response as follows:
        - **Key Points**: List each main point in bullet points.
        - **Summary**: Provide a brief paragraph summarizing the main findings.

        Answer:"""

        inputs = self.tokenizer(prompt, return_tensors="pt", max_length=2048, truncation=True).to(self.model.device)
        outputs = self.model.generate(**inputs, max_new_tokens=50)
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        return response

# Example usage in a Colab environment
def main():
    model_id = "meta-llama/Llama-3.2-3B"  # Updated model ID
    qa_system = PDFQASystem(model_id)

    pdf_path = "/content/temp.pdf"  # Replace with your PDF file path
    if not os.path.exists(pdf_path):
        print("PDF file not found!")
        return

    print("Processing PDF...")
    num_chunks = qa_system.process_pdf(pdf_path)
    print(f"PDF processed into {num_chunks} chunks.")

    while True:
        user_query = input("Enter your question (or type 'exit' to quit): ")
        if user_query.lower() == "exit":
            break

        print("Generating response...")
        response = qa_system.generate_response(user_query)
        print("\nResponse:\n")
        print(response)

if __name__ == "__main__":
    main()


In [ ]:
What is the book about?

In [ ]:
!huggingface-cli login



In [ ]:
import os
from typing import List, Dict
from pypdf import PdfReader
from langchain_core.embeddings import Embeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

class PDFQASystem:
    def __init__(self, model_id: str, device_map: str = "auto"):
        """Initialize the PDF QA System with necessary components."""
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map=device_map
        )
        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2"
        )
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len,
        )
        self.vector_store = None

    def process_pdf(self, pdf_file) -> int:
        """Process a PDF file and create embeddings."""
        pdf_text = self._extract_text_from_pdf(pdf_file)
        chunks = self.text_splitter.split_text(pdf_text)
        self.vector_store = FAISS.from_texts(chunks, self.embeddings)
        return len(chunks)

    def _extract_text_from_pdf(self, pdf_file) -> str:
        """Extract text content from a PDF file."""
        reader = PdfReader(pdf_file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        return text

    def generate_response(self, user_query: str, k: int = 4) -> str:
        """Generate a response using relevant context and the LLM."""
        if not self.vector_store:
            return "Please upload a PDF document first."

        relevant_docs = self.vector_store.similarity_search(user_query, k=k)
        context = "\n".join(doc.page_content for doc in relevant_docs)

        prompt = f"""You are a helpful assistant. Using the following context, please answer the user's question as detailed as possible from the provided context, make sure to provide all the details.
        If the answer cannot be found in the context, say so.

        Context:
        {context}

        Question: {user_query}

        Please structure your response as follows:
        - **Key Points**: List each main point in bullet points.
        - **Summary**: Provide a brief paragraph summarizing the main findings.

        Answer:"""

        inputs = self.tokenizer(prompt, return_tensors="pt", max_length=2048, truncation=True).to(self.model.device)
        outputs = self.model.generate(**inputs, max_new_tokens=300)
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        return response

# Example usage in a Colab environment
def main():
    model_id = "meta-llama/Llama-3.2-3B"  # Updated model ID
    device_map = "auto"  # Specify the device map here
    qa_system = PDFQASystem(model_id, device_map=device_map)

    pdf_path = "sample.pdf"  # Replace with your PDF file path
    if not os.path.exists(pdf_path):
        print("PDF file not found!")
        return

    print("Processing PDF...")
    num_chunks = qa_system.process_pdf(pdf_path)
    print(f"PDF processed into {num_chunks} chunks.")

    while True:
        user_query = input("Enter your question (or type 'exit' to quit): ")
        if user_query.lower() == "exit":
            break

        print("Generating response...")
        response = qa_system.generate_response(user_query)
        print("\nResponse:\n")
        print(response)

if __name__ == "__main__":
    main()

